In [ ]:
# Seq2Seq(Sequence-to-Sequence) 구조 및 동작원리
# 한개의 시퀀스를 다른 시퀀스로 바꾸어주는 딥러닝 모델, 기계 번역 / 질의 응답 / 텍스트 요약과 같은 자연어 처리(NLP) 분약에서 많이 사용되었음

# 기본적인 Seq2Seq 모델은 인코더(encoder)와 디코더(decoder), 그리고 두 블록을 연결하는 컨텍스트 벡터(context vector) 구성되는데,
# 인코더는 입력 시퀀스에 대한 정보를 고정된 길이를 갖는 컨텍스트 벡터로 인코딩(압축)하고,
# 디코더는 컨텍스트 벡터를 이용하여 출력 시퀀스로 디코딩(변환)하는 역할을 수행

# 입력시퀀시 > 인코더(encoder) -> context vector -> 디코더(decoder) > 출력시퀀시
# 기계번역 예시)
# Hello > 인코더(encoder) -> context vector -> 디코더(decoder) > 안녕하세요

# Seq2Seq 전체 아키텍처
# 인코더, 컨텍스트 벡터, 디코더 3개의 블록으로 구성되며, 오차 역전파를 통해서 디코더에서 인코더로 기울기(gradient)가 전달되며 학습이 진행됨
# 그러나 이러한 Seq2Seq 아키텍처는 입력 시퀀스가 짧을때는 그러한 정보를 컨텍스트 벡터에 충분히 담을수 있지만,
# 긴 입력 시퀀스에 대해서는 고정된 컨텍스트 벡터 크기로 인해 정보의 손실이 발생 할 수 밖에 없음

#                                               나는           집에         간다
#                                               softmax   -> softmax  -> softmax
# LSTM -> LSTM -> LSTM ->   context vector ->   LSTM      -> LSTM     -> LSTM
# 임베딩 -> 임베딩 -> 임베딩                         임베딩      -> 임베딩    -> 임베딩
# I -> go -> home                               SOS           나는        집에

In [ ]:
# 인코더(Encoder)
# LSTM, GRU 등을 사용하여 입력시퀀시를 고정된 길이의 벡터로 변환하는 역할 수행
# - 인코더는 입력시퀀시의 각 단어를 순차적으로 처리하면서 각 단계에서 hidden state를 업데이트하고,
# 최종적으로 전체 입력시퀀시를 대표하는 고정 길이의 벡터를 생성함
# 즉 인코더가 인코딩을 한다는 것은 임의의 길이를 가지는 문장을 고정된 길이를 가지는 벡터로 변환하는 작업이라고 할 수 있음

# 'I go home' 문장을 인코더에 입력으로 주면, 인코더는 해당 문장을 인코딩하여 벡터 h를 생성하고 디코더에 전달함
# 인코더가 생성한 벡터 h는 인코더를 이루는 LSTM 마지막 hidden state이고, 중요한 점은 벡터 h는 고정된 길이를 가지는 벡터라는 것임
# 임베딩(Embedding): 자연어처리에서 사람이 쓰는 자연어를 컴퓨터가 이해할 수 있도록 숫자 형태인 vector로 바꾸는 과정
# 예를 들면 'I' -> [0.1,-0.5,0.2,0.3], 'go' -> [0.4,0.9,-0.2,-0.7], 'home' -> [0.6,-0.8,-0.7,0.9]

# 예시)
# LSTM -> LSTM -> LSTM -> h -> context vector
# 임베팅 -> 임베딩 -> 임베딩 -> h - context vector
# I -> go -> home -> h - context vector 

In [ ]:
# 디코더(Decoder)
# 인코더 출력인 고정 길이 벡터를 기반으로 원하는 출력 시퀀스를 생성하는 역할을 수행
# 디코더 역시 LSTM, GRU 등의 구조를 이용하는데, 일반적인 LSTM, GRU 구조와 다른 점은 hidden state 초기값이 컨텍스트 벡터 h 라는 것임,
# 또한 디코더 예측값은 softmax를 통해 가장 확률이 높은 단어를 선택하면서 최종적으로 우리가 원하는 출력 시퀀스 만들어감
# SOS: 출력 시퀀스의 시작을 나타내는데 사용되는 것으로 Start Of Sequence를 의미함
# 예시)
#                    나는           집에         간다
#                   softmax   -> softmax  -> softmax
# context vector -> LSTM      -> LSTM     -> LSTM
#                   임베딩      -> 임베딩    -> 임베딩
#                   SOS           나는        집에
# 디코더는 인코더의 마지막 hidden state인 컨텍스트 벡터를 첫번째 hidden state 값으로 사용
# 1)디코더의 첫번째 LSTM 셀은, 
# 컨텍스트 벡터인 첫번째 hidden state 값과 현재 t 시점에서의 입력값인 SOS 로부터 다음에 등장할 단어 '나는' 을 예측하고,
# 2)예측된 '나는' t + 1 시점의 입력값이 되어서 다음 단어 '집에'를 예측하고
# 3)예측된 '집에'와 t + 1 시점의 LSTM의 hidden state 값이 결합되어 최종적으로 '간다'를 예측함